## Turn this into a good-looking-informative-cell
#### The output .csv file will contain the following fields
    - channelGrouping
    - fullVisitorId
    - visitId
    - visitNumber
    - visitStartTime
    - browser
    - deviceCategory
    - operatingSystem
    - city
    - country
    - region
    - subContinent
    - hits
    - newVisits
    - *timeOnSite*
    - totalTransactionRevenue
    - *transactions dividir esto*
    - adHasContent
    - adPosition
    - campaign
    - isTrueDirect
    - medium
    - source

### Funciones usadas en el Notebook

In [12]:
# recursive flatten function from https://towardsdatascience.com/flattening-json-objects-in-python-f5343c794b10
# Call example:
  # flattenCol = [flatten_json(json.loads(d)) for d in df['trafficSource']]
  # dataFrameOfFlattenCol = pd.DataFrame(flattenCol)
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

def region_country(geo):
    if geo['region'] == '(not set)':
        return geo['country']
    elif geo['region'] == 'not available in demo dataset':
        return geo['country']
    else:
        return geo['region']

### Importación de librerías y datos (separándolos a través de comandos bash)

In [11]:
import pandas as pd
import glob
import json
import numpy as np

pd.set_option('display.max_columns', 30)

/bin/sh: line 0: cd: datasetPath: No such file or directory


In [13]:
# TO DO: merge these two cells

# bash commands for splitting a huge dataset into several smaller ones (~1.5GB size each)
!rm -r -f ../data/tempSplittedData
!mkdir ../data/tempSplittedData
!mv ../data/train_v2.csv ../data/tempSplittedData
!cd ../data/tempSplittedData/ && head -n 1 train_v2.csv > headerfile
!cd ../data/tempSplittedData && tail -n +2 train_v2.csv > train_v2.tmp && mv train_v2.tmp train_v2.csv
!cd ../data/tempSplittedData && split -l 100000 --additional-suffix=.csv train_v2.csv
!for csv! in *.csv; do cat headerfile $csv > tmpfile2; mv tmpfile2 $csv; done
!mv ../data/tempSplittedData/train_v2.csv ../data/
!rm ../data/tempSplittedData/headerfile

^C
split: illegal option -- -
usage: split [-a sufflen] [-b byte_count] [-l line_count] [-p pattern]
             [file [prefix]]
/bin/sh: `csv!': not a valid identifier


In [14]:
!cd ../data/ && cut -d, -fhits --complement train_v2.csv 

cut: [-cf] list: illegal list value


In [37]:
allFiles = glob.glob("../data/tempSplittedData/*.csv")
df = pd.DataFrame({'channelGrouping': [],
                    'device': [],
                    'fullVisitorId': [],
                    'geoNetwork': [],
                    'socialEngagementType': [],
                    'totals': [],
                    'trafficSource': [],
                    'visitNumber': [],
                    'visitStartTime': []})

for fl in allFiles:
    temp = pd.read_csv(fl, usecols=["channelGrouping", "date", "device", "fullVisitorId", "geoNetwork",
                                    "socialEngagementType", "totals", "trafficSource", "visitId", "visitNumber",
                                    "visitStartTime"], header=0, dtype={'fullVisitorId': 'str'})
    # we leave out hits, customDimensions and date
    temp.drop(columns=["date", "visitId"], axis=1, inplace=True)
    df.append(temp, ignore_index=True, inplace=True)

# remove the splitted files
!rm -r "../data/tempSplittedData"

In [38]:
df.describe(include='all')

,channelGrouping,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitNumber,visitStartTime
count,1708337,1708337,1708337,1708337,1708337,1708337,1708337,1.708337e+06,1.708337e+06
unique,8,298,1323730,59955,1,252374,76637,NaN,NaN
top,Organic Search,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1957458976293878100,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",NaN,NaN
freq,738963,504589,400,132962,1708337,350177,460901,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.335170e+00,1.498352e+09
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.354034e+00,1.624937e+07
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00,1.470035e+09
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00,1.482738e+09
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00,1.499832e+09
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00,1.512513e+09


In [39]:
df.head(5)

,channelGrouping,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitNumber,visitStartTime
0,Organic Search,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1.0,1.508198e+09
1,Referral,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",6.0,1.508176e+09
2,Direct,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1.0,1.508202e+09
3,Organic Search,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1.0,1.508170e+09
4,Organic Search,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1.0,1.508191e+09


In [40]:
df.shape

(1708337, 9)

In [41]:
# df = df.replace({
# MERGE    "(not set)": np.nan,                  "(none)": np.nan,
#               KEEP   "not available in demo dataset": np.nan})

De las columnas unidimensionales mantenemos:
- fullVisitorId
- visitStartTime
- channelGrouping

In [2]:
df["visitStartTime"] = pd.to_datetime(df["visitStartTime"], unit='s')
df.drop(['socialEngagementType'], axis=1, inplace=True)

NameError: name 'pd' is not defined

De la columna 'device' mantenemos:
- browser
- deviceCategory
- operatingSystem

In [43]:
deviceFlatten = [flatten_json(json.loads(d)) for d in df['device']]
devi = pd.DataFrame(deviceFlatten)

devi.drop(columns=['browserSize', 'browserVersion', 'flashVersion', 'language', 
    'mobileDeviceBranding', 'mobileDeviceInfo', 'mobileDeviceMarketingName',
    'mobileDeviceModel', 'mobileInputSelector', 'operatingSystemVersion',
    'screenColors','screenResolution'], inplace=True)

#we map our data to its corresponding value
isMobile = {False : 0, True : 1}
devi["isMobile"] = devi["isMobile"].map(isMobile)

devi.drop(columns=['isMobile'], inplace=True, axis=1)
df.drop(columns=['device'], inplace=True, axis=1)
df = df.join(devi)

['channelGrouping', 'fullVisitorId', 'geoNetwork', 'totals', 'trafficSource', 'visitNumber', 'visitStartTime', 'browser', 'deviceCategory', 'operatingSystem']


De la columna 'geoNetwork' mantenemos:
- subcontinent
- coutry
- region (que contiene ciudad si no está disponible)
- city

In [44]:
geoNetworkFlatten = [flatten_json(json.loads(d)) for d in df['geoNetwork']]
geo = pd.DataFrame(geoNetworkFlatten)
geo.drop(['latitude','longitude','networkLocation','cityId','continent','metro', 'networkDomain'], axis=1, inplace=True)

geo['region']=geo.apply(region_country,axis=1)
# geo.loc[geo['city'] == "not available in demo dataset",'city'] = np.nan

# geo['region'].fillna(geo['country'])

df.drop(columns=['geoNetwork'], inplace=True, axis=1)
df = df.join(geo)

De la columna 'totals' mantenemos:
- hits
- transactionRevenue
- newVisits

In [46]:
#Transform json
totalsList = [flatten_json(json.loads(d)) for d in df['totals']]
totals_df = pd.DataFrame(totalsList)

#Remove visits column as it does not provide any information
# Remove sessionQualityDim: An estimate of how close a particular session was to transacting, ranging
    # from 1 to 100, calculated for each session. A value closer to 1 indicates a low session quality, or
    # far from transacting, while a value closer to 100 indicates a high session quality, or very close to
    # transacting. A value of 0 indicates that Session Quality is not calculated for the selected time range.
# Remove transactionRevenue (deprecated) to use totalTransactionRevenue instead
totals_df = totals_df.drop(['visits', 'sessionQualityDim', 'transactionRevenue', 'hits', 'newVisits'], axis=1)

#Change nan in "transactionRevenue" for 0. 
totals_df['totalTransactionRevenue'] = totals_df['totalTransactionRevenue'].fillna(0)

#Change nan in "bounces" for 0
totals_df['bounces'] = totals_df['bounces'].fillna(0)

#There are some nan in pageviews but it is not trivial to find a value we can change these nan by logically.
#The corrlation with hits is 0.984, so we can keep hits that does not have any nan value. Keeping both could be redundant.
totals_df = totals_df.drop(['pageviews'], axis=1)

#Not 100% proven yet, but i have a big feeling that bounces is mainly 1 when there is only 1 hit by defenition.
#This means that even though the correlation doesn't provide a big value beetween them two, bounces is not giving much 
#additional info. Meaning using it is not relevant.
totals_df = totals_df.drop(['bounces'], axis=1)

totals_df['totalTransactionRevenue'] = totals_df['totalTransactionRevenue'].apply(lambda x: float(x)/1000000)

df.drop(['totals'], inplace=True, axis=1)
df = df.join(totals_df)

To be removed:

In [47]:
trafficFlatten = [flatten_json(json.loads(d)) for d in df['trafficSource']]
traffic = pd.DataFrame(trafficFlatten)

In [48]:
workingdf = traffic.copy()

workingdf.drop(['adwordsClickInfo_criteriaParameters', 'adwordsClickInfo_adNetworkType',
    'adwordsClickInfo_gclId', 'adwordsClickInfo_isVideoAd', 'adwordsClickInfo_page',
    'keyword', 'referralPath'], axis=1, inplace=True)

# workingdf['campaign'].value_counts()

# workingdf.sample(10)
# workingdf.describe()
workingdf['source'].value_counts()

google                         658384
(direct)                       565975
youtube.com                    329450
analytics.google.com            37436
Partners                        32931
sites.google.com                 9072
google.com                       8341
m.facebook.com                   6737
dfa                              6704
baidu                            4971
reddit.com                       4092
facebook.com                     3584
googleads.g.doubleclick.net      3539
qiita.com                        3394
bing                             3368
quora.com                        3032
mail.google.com                  2558
yahoo                            2436
siliconvalley.about.com          2097
groups.google.com                2009
t.co                             2005
l.facebook.com                   1529
blog.golang.org                  1382
dealspotr.com                     750
support.google.com                676
productforums.google.com          672
plus.google.

In [50]:
# df['campaign'].value_counts()

In [51]:
workingdf['campaign'].value_counts()

(not set)                                                               1604526
Data Share Promo                                                          32914
1000557 | GA | US | en | Hybrid | GDN Text+Banner | AS                    24410
1000557 | GA | US | en | Hybrid | GDN Remarketing                         15149
AW - Dynamic Search Ads Whole Site                                        15146
AW - Accessories                                                           7972
Smart Display Campaign                                                     2664
"google + redesign/Accessories March 17" All Users Similar Audiences       1179
Page: contains "/google+redesign/drinkware" Similar Audiences               611
"google + redesign/Accessories March 17" All Users                          562
Sports & Fitness/Health & Fitness Buffs                                     463
Run of Network Line Item                                                    428
Media & Entertainment/Movie Lovers      

/To be removed>

De la columna 'trafficSource' mantenemos:
- adContent
- adPosition
- campaign
- isTrueDirect
- medium
- source

In [56]:
trafficFlatten = [flatten_json(json.loads(d)) for d in df['trafficSource']]
traffic = pd.DataFrame(trafficFlatten)
traffic.drop(['adwordsClickInfo_criteriaParameters', 'adwordsClickInfo_adNetworkType',
    'adwordsClickInfo_gclId', 'adwordsClickInfo_isVideoAd', 'adwordsClickInfo_page',
    'keyword', 'referralPath'], axis=1, inplace=True)

# rename to a simpler name
traffic.rename(index=str, columns={
    "adwordsClickInfo_slot": "adPosition"}, inplace=True)

df['adContent'] = traffic['adContent'].copy()
df['adPosition'] = traffic['adPosition'].copy()
df['campaign'] = traffic['campaign'].copy()
df['isTrueDirect'] = traffic['isTrueDirect'].copy()
df['medium'] = traffic['medium'].copy()
df['source'] = traffic['source'].copy()

# TO DO: traffic['source'] regex for the main domain (last whatevs.com), and drop less than 1000
# traffic['source'] = traffic['source'].str.extract('{"browser": "(.+?)"', expand=True)

df.drop(columns=['trafficSource'], inplace=True, axis=1)
df = traffic.join(df)

In [93]:
# df.drop(columns=['campaign'], inplace=True, axis=1)
# df['campaign'] = traffic['campaign'].copy()
# df['position'] = traffic['adPosition'].copy()
# df['position'].value_counts()

De trafficSource mantenemos:
- adContent
- adPosition
- campaign
- isTrueDirect
- medium
- source

In [96]:
# a = traffic.join(df)

In [98]:
# df = a.copy()

In [4]:
# df['adContent'].value_counts()

In [ ]:
# df = df.replace({"(not set)": np.nan,
#                  "(none)": np.nan,
#                  "not available in demo dataset": np.nan})

In [99]:
list(df)

['adContent',
 'adPosition',
 'campaign',
 'campaignCode',
 'isTrueDirect',
 'medium',
 'source',
 'channelGrouping',
 'fullVisitorId',
 'trafficSource',
 'visitNumber',
 'visitStartTime',
 'browser',
 'deviceCategory',
 'operatingSystem',
 'city',
 'country',
 'region',
 'subContinent',
 'timeOnSite',
 'totalTransactionRevenue',
 'transactions',
 'position']

In [101]:
print(df.shape)

(1708337, 23)
64737


In [ ]:
df.describe(include='all')

In [ ]:
df.head(5)

In [103]:
df.to_csv("../data/test_v2_cleaned.csv")
df.to_pickle("../data/test_v2_cleaned.pkl")